In [1]:
import random

In [2]:
def are_valid_clause_variables(clause, num_variables):
    """Checks if a clause has valid variables (i.e. is consistent with the number of variables given)

    Parameters
    ----------
    clause: str
        the clause to check
    num_variables: int
        the number of total variables
    
    Returns
    -------
    bool
        True if the clause is consistent, False otherwise
    """

    for elem in clause:
        if clause.count(elem) > 1 or (elem in clause and elem + num_variables in clause):
            return False
    return True

In [3]:
def create_clause(list_of_variables, num_variables):
    """Creates a random clause

    Parameters
    ----------
    list_of_variables: list(int)
        the variables to choose from
    num_variables: int
        the number of variables in total
    
    Returns
    -------
    str
        the resulting clause
    """

    clause = ""
    random_clause_variables = [0, 0, 0]
    while not are_valid_clause_variables(random_clause_variables, num_variables):
        random_clause_variables = random.sample(list_of_variables, 3)
        random_clause_variables.sort()
    
    if random_clause_variables[0] >= num_variables:
        clause += f"n{random_clause_variables[0]-num_variables}"
    else:
        clause += f"{random_clause_variables[0]}"

    if random_clause_variables[1] >= num_variables:
        clause += f" n{random_clause_variables[1]-num_variables}"
    else:
        clause += f" {random_clause_variables[1]}"

    if random_clause_variables[2] >= num_variables:
        clause += f" n{random_clause_variables[2]-num_variables}"
    else:
        clause += f" {random_clause_variables[2]}"

    return clause

In [4]:
def exists(clause_to_check, set_to_check):
    """Checks if a clause already exists in a set of clauses

    Parameters
    ----------
    clause_to_check: str
        the input clause
    set_to_check: set(str)
        the set to check if clause_to_check is contained or not
    
    Returns
    -------
    bool
        True if the set contains the clause, False otherwise
    """

    if clause_to_check == None:
        return True
    for clause in set_to_check:
        count = 0
        [v1, v2, v3] = clause.split(" ")
        for variable in clause_to_check.split(" "):
            if v1 == variable or v2 == variable or v3 == variable:
                count += 1
        if count == 3:
            return True
    return False

In [5]:
def fac(n):
    """Calculates the factorial of a number

    Parameters
    ----------
    n: int
        the input number
    
    Returns
    -------
    int
        the result
    """
    
    if n == 0 or n == 1:
        return 1
    else:
        return n * fac(n-1)

In [6]:
def n_choose_k(n, k) -> int:
    """Calculates N choose K for N and K integers

    Parameters
    ----------
    n: int
        N
    k: int
        K
    
    Returns
    -------
    int
        the result
    """
    
    return int(fac(n)/(fac(k)*fac(n-k)))

In [7]:
def create_cnf(num_variables, num_clauses, distinct_clauses=False):
    """Creates a random CNF

    Parameters
    ----------
    num_variables: int
        the number of variables the CNF should have
    num_clauses: int
        the number of clauses the CNF should have
    distinct_clauses: bool (default=False)
        True if we want to force distinct clauses, False otherwise
    
    Returns
    -------
    str
        the created CNF
    """

    if num_variables < 3:
        print("ERROR: Not possible with num_var < 3")
        return None

    if distinct_clauses and num_clauses > n_choose_k(2*num_variables, 3):
        print("ERROR: Not possible. Decrease the number of clauses or increase the number of variables")
        return None
    cnf = ""
    list_of_variables = list(range(2*num_variables))
    existing_clauses = set()
    for _ in range(num_clauses-1):
        created_clause = None
        if distinct_clauses:
            while created_clause == None or exists(created_clause, existing_clauses):
                created_clause = create_clause(list_of_variables, num_variables)
        else:
            created_clause = create_clause(list_of_variables, num_variables)
        cnf += f"{created_clause},"
        existing_clauses.add(created_clause)
    
    created_clause = None
    if distinct_clauses:
        while created_clause == None or exists(created_clause, existing_clauses):
            created_clause = create_clause(list_of_variables, num_variables)
    else:
        created_clause = create_clause(list_of_variables, num_variables)
    cnf += f"{created_clause}"
    return cnf

In [ ]:
# Example of creating a CNF with 3 variables and 8 clauses
print(create_cnf(3, 8, distinct_clauses=True))

In [9]:
def satisfies_cnf_nae(candidate_solution, cnf, num_variables):
    """Checks if a candidate solution satisfies a CNF or not with NAE constraint

    Parameters
    ----------
    candidate_solution: list(int)
        the solution to test
    cnf: str
        the cnf in question
    num_variables: int
        the number of variables in the CNF
    
    Returns
    -------
    bool
        True if the candidate solution satisfies the CNF, False otherwise
    """

    clauses = cnf.split(",")
    for clause in clauses:
        clause_vars = clause.split(" ")
        clause_result = False
        for i in range(len(clause_vars)):
            if clause_vars[i].startswith("n"):
                clause_vars[i] = not candidate_solution[int(clause_vars[i][1:])-num_variables]
            else:
                clause_vars[i] = candidate_solution[int(clause_vars[i])]
            clause_result += clause_vars[i]
        if clause_result == 3 or clause_result == 0:
            return False
    return True


In [10]:
def satisfiable_or_not(num_variables, cnf):
    """Classically solves a NAE-3SAT problem on a CNF

    Parameters
    ----------
    num_variables: int
        the number of variables in the CNF
    cnf: str
        the input CNF

    Returns
    -------
    (bool, list(str))
        if True (i.e. the CNF is satisfiable), then the list contains the solution. Empty if the CNF is not satisfiable 
    
    """

    num_possible_combinations = 2**num_variables
    for i in range(num_possible_combinations):
        binary_string = format(i, 'b').zfill(num_variables)
        candidate_solution = list(map(int, list(binary_string)))
        if satisfies_cnf_nae(candidate_solution, cnf, num_variables):
            return True, candidate_solution
    return False, []

In [11]:
"""
Generate random NAE-3SAT and non-NAE-3SAT CNFs
"""

nae3sat_cnfs_file = open("nae3sat_cnfs_20vars.txt", "w")
non_nae3sat_cnfs_file = open("non-nae3sat_cnfs_20vars.txt", "w")
max_vars = 21
for num_vars in range(3, max_vars):
    for num_clauses in range(num_vars, 3*num_vars):
        for i in range(10):
            cnf = create_cnf(num_vars, num_clauses, distinct_clauses=True)
            print(f"For {num_vars}/{max_vars} on {num_clauses}/{n_choose_k(2*num_vars, 3)}", end="\r")
            if cnf != None:
                satisfiable, solution = satisfiable_or_not(num_vars, cnf)
                if satisfiable:
                    nae3sat_cnfs_file.write(f"{num_vars} -- {cnf}\n")
                else:
                    non_nae3sat_cnfs_file.write(f"{num_vars} -- {cnf}\n")
nae3sat_cnfs_file.close()
non_nae3sat_cnfs_file.close()